In [11]:
sys.path.insert(0, '/Users/eeko/openms-develop/openms_build/pyOpenMS')
from pyopenms import *
import os
import glob

In [12]:
path= "results/interim"
if not os.path.exists(path):
    os.mkdir(path)

#### `1) Feature detection `
This series of algorithm is used for mass trace detection, chromatographic deconvolution of co-eluting peaks and data reduction.

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/TOPP_FeatureFinderMetabo.html

In [13]:
input_mzml_files = glob.glob('Example_data/*.mzML')

# 1.1) Mass trace detection

for filename in input_mzml_files:
    print("Mass Trace Detection: ", filename)
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    mass_traces = []
    mtd = MassTraceDetection()
    mtd_par = mtd.getDefaults()
    mtd_par.setValue("mass_error_ppm", 10.0) 
    mtd_par.setValue("noise_threshold_int", 1.0e04)
    mtd.setParameters(mtd_par)
    mtd.run(exp, mass_traces, 0)

# 1.2) Elution peak detection
    print("Elution Peak Detection: ", filename)
    mass_traces_split = []
    mass_traces_final = []
    epd = ElutionPeakDetection()
    epd_par = epd.getDefaults()
    epd_par.setValue("width_filtering", "fixed")
    epd.setParameters(epd_par)
    epd.detectPeaks(mass_traces, mass_traces_split)
     
    if (epd.getParameters().getValue("width_filtering") == "auto"):
          epd.filterByPeakWidth(mass_traces_split, mass_traces_final)
    else:
          mass_traces_final = mass_traces_split

# 1.3) Feature detection
    print("Feature Detection: ", filename)
    feature_map_FFM = FeatureMap()
    feat_chrom = []
    ffm = FeatureFindingMetabo()
    ffm_par = ffm.getDefaults() 
    ffm_par.setValue("isotope_filtering_model", "none")
    ffm_par.setValue("remove_single_traces", "true")
    ffm_par.setValue("mz_scoring_by_elements", "false")
    ffm_par.setValue("report_convex_hulls", "true")
    ffm.setParameters(ffm_par)
    ffm.run(mass_traces_final, feature_map_FFM, feat_chrom)
    feature_map_FFM.setUniqueIds()
    feature_map_FFM.setPrimaryMSRunPath([filename.encode()])
    print(filename[7:-5] + ".featureXML")
    FeatureXMLFile().store(os.path.join(path, os.path.basename(filename)[:-5] + ".featureXML"), feature_map_FFM)
    
print("Finished Feature Detection")

Mass Trace Detection:  Example_data/Actino_1.mzML
Progress of 'mass trace detection':
Elution Peak Detection:  Example_data/Actino_1.mzML
-- done [took 0.10 s (CPU), 0.10 s (Wall)] -- 
 Example_data/Actino_1.mzML
_data/Actino_1.featureXMLProgress of 'elution peak detection':
-- done [took 0.55 s (CPU), 0.55 s (Wall)] -- 
Progress of 'assembling mass traces to features':
-- done [took 0.45 s (CPU), 0.45 s (Wall)] -- 

Mass Trace Detection:  Example_data/Actino_2.mzML
Progress of 'mass trace detection':
-- done [took 0.06 s (CPU), 0.06 s (Wall)] -- 
 Example_data/Actino_2.mzML
Feature Detection: Progress of 'elution peak detection':
-- done [took 0.30 s (CPU), 0.29 s (Wall)] -- 
 Example_data/Actino_2.mzML
Progress of 'assembling mass traces to features':
-- done [took 0.17 s (CPU), 0.18 s (Wall)] -- 
_data/Actino_2.featureXML
Finished Feature Detection


In [14]:
input_features = sorted(glob.glob('results/interim/*.featureXML'))

for consensus in input_features:
    fmap = FeatureMap()
    FeatureXMLFile().load(consensus, fmap)
    DF= fmap.get_df()
    DF= DF.drop(columns=["ID_native_id", "peptide_score", "peptide_sequence", "ID_filename"])
    DF=DF.rename(columns={"f1":"intensity"})
print("example:", os.path.basename(consensus))
display(DF)

example: Actino_2.featureXML


,charge,RT,mz,RTstart,RTend,MZstart,MZend,quality,intensity
feature_id,,,,,,,,,
7567800095073218753,1,196.657536,152.070631,186.985717,379.290435,152.070541,153.073975,0.000116,1.835233e+06
7887145815028143368,1,89.349217,155.081511,43.833829,610.517561,155.080276,156.085022,0.000350,5.567450e+06
10047496411598284682,1,186.985717,160.075681,157.853123,327.955560,160.075623,161.079056,0.000363,5.690979e+06
15902010377849777468,1,124.456095,160.075696,45.542578,156.684687,160.074478,161.079178,0.000381,5.925535e+06
9497790646127226659,1,302.534687,162.054991,292.990932,324.362132,162.054855,163.058319,0.000762,1.217674e+07
...,...,...,...,...,...,...,...,...,...
13284859976832795220,1,53.012203,1477.003140,50.455839,57.913036,1477.001343,1478.005371,0.000029,2.604481e+05
353591387967773467,1,51.728465,1477.497953,50.455839,61.539083,1477.496826,1479.508179,0.000095,9.533261e+05
5090387812630044532,2,61.539083,1485.510259,59.091381,65.314499,1485.504517,1486.517456,0.000072,3.911082e+05


#### `2) MapAlignerPoseClustering `
This algorithm is used to perform a linear retention time alignment, in order to correct for chromatographic shifts in retention time. The reference file used for Map Alignment is the feature map with the highest number of features.

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_MapAlignerPoseClustering.html

In [18]:
path= "results/interim/"
isExist= os.path.exists(path)
if not isExist:
    os.mkdir(path)

input_feature_files = sorted(glob.glob('results/interim/*.featureXML'))
feature_maps=[]
for filename in input_feature_files:
    feature_map_MFD = FeatureMap()
    FeatureXMLFile().load(filename, feature_map_MFD)
    feature_maps.append(feature_map_MFD)

ref_index = [i[0] for i in sorted(enumerate([fm.size() for fm in feature_maps]), key=lambda x:x[1])][-1]

aligner = MapAlignmentAlgorithmPoseClustering()
aligner_par= aligner.getDefaults()

aligner_par.setValue("max_num_peaks_considered", -1)
aligner_par.setValue("superimposer:mz_pair_max_distance", 0.05)
aligner_par.setValue("pairfinder:distance_MZ:max_difference", 10.0)
aligner_par.setValue("pairfinder:distance_MZ:unit", "ppm")
aligner.setParameters(aligner_par)
aligner.setReference(feature_maps[ref_index])

for feature_map in feature_maps[:ref_index] + feature_maps[ref_index+1:]:
    trafo = TransformationDescription()
    aligner.align(feature_map, trafo)
    transformer = MapAlignmentTransformer()
    transformer.transformRetentionTimes(feature_map, trafo, True) # store original RT as meta value

for feature_map in feature_maps:    
    feature_file = os.path.join(path, 'MapAligned_' + os.path.basename(feature_map.getMetaValue('spectra_data')[0].decode())[:-5] +".featureXML")
    FeatureXMLFile().store(feature_file, feature_map)

#### `3) FeatureGroupingAlgorithmKD `

Feature linker clusters the feature information (from single files) into a ConsensusFeature, linking features from different files together, which have a smiliar m/z and rt (no MS2 data).

###### Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_FeatureLinkerUnlabeledKD.html

In [20]:
input_feature_files = sorted(glob.glob("results/interim/MapAligned*.featureXML"))

feature_maps = []
for featurexml_file in input_feature_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)

In [21]:
feature_grouper = FeatureGroupingAlgorithmKD()

consensus_map = ConsensusMap()
file_descriptions = consensus_map.getColumnHeaders()

for i, feature_map in enumerate(feature_maps):
    file_description = file_descriptions.get(i, ColumnHeader())
    file_description.filename = os.path.basename(feature_map.getMetaValue('spectra_data')[0].decode())[7:]
    file_description.size = feature_map.size()
    file_descriptions[i] = file_description

feature_grouper.group(feature_maps, consensus_map)
consensus_map.setColumnHeaders(file_descriptions)


Consensus_file= os.path.join(path, 'consensus' + ".consensusXML")
ConsensusXMLFile().store(Consensus_file, consensus_map)


# get intensities as a DataFrame
result = consensus_map.get_df()
result= result.reset_index()
result= result.drop(columns= ["id", "sequence", "quality"])
# store as tsv file
result.to_csv('results/FeatureMatrix.tsv', sep = '\t', index = False)
result

Progress of 'computing RT transformations':
-- done [took 0.01 s (CPU), 0.01 s (Wall)] -- 
Progress of 'linking features':
-- done [took 0.02 s (CPU), 0.02 s (Wall)] -- 
ConsensusXMLFile::store():  found 2194 invalid unique ids


,charge,RT,mz,1.mzML,2.mzML
0,1,528.602024,432.238050,1.502651e+06,1.736886e+07
1,1,72.240268,268.104015,3.684163e+05,8.183538e+06
2,2,63.104790,1242.434351,7.907504e+05,3.215331e+05
3,1,499.901992,387.180417,5.935919e+08,9.093218e+07
4,1,80.627223,212.102916,7.761065e+06,5.894826e+06
...,...,...,...,...,...
2188,1,546.385270,302.232530,0.000000e+00,2.963920e+06
2189,1,236.110587,526.181924,0.000000e+00,1.088353e+06
2190,2,375.704452,582.356078,0.000000e+00,5.372851e+05
2191,2,395.639263,973.902588,0.000000e+00,1.009125e+07
